In [11]:
from graphviz import Digraph
import numpy as np
import matplotlib.pyplot as plt

def plot_tree_with_path(start, path_nodes):
    dot = Digraph(comment='Operations Tree')

    def add_edges(node):
        node_id = f'{node.value}_{id(node)}'
        if node not in path_nodes:
            dot.node(node_id, str(node.value))
        else:
            dot.node(node_id, str(node.value), color='green')
        for child in node.children:
            child_id = f'{child.value}_{id(child)}'
            dot.edge(node_id, child_id, label=child.prev_op)
            add_edges(child)

    add_edges(start)
    dot.render('tree_output', view=True)


In [17]:
class TNode:
    def __init__(self, parent, value, prev_op):
        self.parent = parent
        self.value = value
        self.prev_op = prev_op
        self.children = []
    
    def add_child(self, child):
        self.children.append(child)

class QNode:
    def __init__(self, value):
        self.value = value
        self.next = None

class Queue:
    def __init__(self):
        self.front = None
        self.rear = None

    def is_empty(self):
        return self.front is None

    def enqueue(self, value):
        new_node = QNode(value)
        if self.rear is None:
            self.front = self.rear = new_node
            return
        self.rear.next = new_node
        self.rear = new_node

    def dequeue(self):
        if self.is_empty():
            raise Exception("Trying to dequeue from an empty queue")
        temp = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        return temp.value

    def peek(self):
        if self.is_empty():
            raise Exception("Trying to peek from an empty queue")
        return self.front.value

class BitVector:
    def __init__(self, size):
        self.size = size
        self.bits = [0] * ((size + 7) // 8)

    def set(self, index):
        byte_index = index // 8
        bit_index = index % 8
        self.bits[byte_index] |= (1 << bit_index)

    def check(self, index):
        byte_index = index // 8
        bit_index = index % 8
        return (self.bits[byte_index] & (1 << bit_index)) != 0

def op_a(value, Y, m):
    return min(value * m, Y * 2), 'a'

def op_b(value):
    return value - 2 , 'b'

def op_c(value):
    return value - 1 , 'c'


def from_x_to_y(X, Y, m):
    results = BitVector(1000 * X * m)
    operations = [lambda x: op_a(x, Y, m), op_b, op_c]
    
    q = Queue()
    start = TNode(None, X, None)
    q.enqueue(start)
    results.set(start.value)
    
    path_nodes = set()
    
    if start.value == Y:
        path_nodes.add(start)
        return start, start, path_nodes
    else:
        while not q.is_empty():
            cur = q.dequeue()
            
            for op in operations:
                res, name = op(cur.value)
                
                if res == Y:
                    result_node = TNode(cur, res, name)
                    cur.add_child(result_node)
                    path_nodes.add(result_node)
                    
                    while cur:
                        path_nodes.add(cur)
                        cur = cur.parent
                        
                    return result_node, start, path_nodes
                
                if not results.check(res):
                    results.set(res)
                    node = TNode(cur, res, name)
                    cur.add_child(node)
                    q.enqueue(node)
                    
    return -1, start, path_nodes




def build_path(end_node):
    path = []
    current_node = end_node
    while current_node is not None:
        if current_node.prev_op is not None:
            path.append((current_node.value, current_node.prev_op))
        else:
            path.append((current_node.value, 'Start'))
        current_node = current_node.parent

    path.reverse()
    return path


res, start, path_nodes = from_x_to_y(54, 29, 3)
if res != -1:
    full_path = build_path(res)
    print("Path from start to exit:", full_path)
    plot_tree_with_path(start, path_nodes)
else:
    print("No path found.")

Path from start to exit: [(54, 'Start'), (52, 'b'), (50, 'b'), (48, 'b'), (46, 'b'), (44, 'b'), (42, 'b'), (40, 'b'), (38, 'b'), (36, 'b'), (34, 'b'), (32, 'b'), (30, 'b'), (29, 'c')]
